### Вопросы на будущее

Насколько сложная структура может быть (насколько сложные и разнообразные документы)? 

Как оценивать вместе с номером уровня заголовка?


## Извлечение текста из pdf
https://www.severcart.ru/blog/all/tesseract_ocr_python/

In [ ]:
from pdf2text import pdf2text

## Классификация заголовков

In [1]:
import re
RE_LIST = re.compile(r'\d+(\.\d+)*\D') # для отдельного типа списка
RE_HEADER = re.compile(r'Раздел|Подраздел|Глава|Параграф|Секция|Часть|Статья')

In [2]:
import pytesseract

def mean_bbox_size(img):
    """
    returns (mean_height, mean_width)
    """
    d = pytesseract.image_to_data(img, lang='rus+eng', 
                                  output_type=pytesseract.Output.DICT)
    
    box = [0, 0, 0] # heights, widths, num_lines
    for i in range(len(d['level'])):
        if d['level'][i] == 4:  # bounding box of text line
            box[0] += d['height'][i]
            box[1] += d['width'][i]
            box[2] += 1
    return (box[0] / box[2], box[1] / box[2])

In [11]:
import cv2

class AddImgFeatures:
    def __init__(self):
        pass
    def fit(self):
        pass
    def transform(self, X):
        """
        X - dict {"text": "", "bbox": [], "name": ""}
        computes mean size of bbox
        returns features normalized bbox sizes + mean size of bbox
        """
        path = 'docs'
        mean_heights = {} # {"name": ["mean_height", "mean_width", "height", "width"]}
        features = []
        for elem in X:
            if elem['name'] not in mean_heights: 
                img = cv2.imread(path + '/' + elem['name'])
                mean_height, mean_width = mean_bbox_size(img)
                mean_heights[elem['name']] = [mean_height / img.shape[0], 
                                              mean_width / img.shape[1], 
                                              img.shape[0], img.shape[1]]
            h0 = mean_heights[elem['name']][2]
            w0 = mean_heights[elem['name']][3]
            # normalized left, normalized top?,
            # normalized width, normalized height,
            # mean height, mean width
            features.append([elem['bbox'][0] / w0, #elem['bbox'][1] / h0,
                             elem['bbox'][2] / w0, elem['bbox'][3] / h0,
                             mean_heights[elem['name']][0], 
                             mean_heights[elem['name']][1]])
        return features
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [4]:
class AddFeatures:
    def __init__(self):
        pass
    def fit(self):
        pass
    def transform(self, X):
        """
        features 2 columns: 1-list, 2-header
        X - dict {"text": "", "bbox": [], "name": ""}
        """
        features = []
        for elem in X:
            line = elem['text']
            match = RE_LIST.search(line)
            if match:
                if match.start() == 0:
                    features.append([1, 0])
                    continue
            match = RE_HEADER.search(line)
            if match:
                if match.start() == 0:
                    features.append([0, 1])
                    continue
            features.append([0, 0])
        return features
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [5]:
class string2features:
    def __init__(self):
        pass

    def fit(self):
        pass

    def predict(self):
        pass

    def fit_transform(self, X, y):
        return self.transform(X)

    def transform(self, X):
        """
        X - список строк
        """
        first_words = []
        for elem in X:
            line = elem['text']
            line_words = line.split()
            if len(line_words) > 1:
                first_words.append(line_words[0] + ' ' + line_words[1])
            elif line_words:
                first_words.append(line_words[0])
            else:
                first_words.append('')
        return first_words

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [6]:
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
#from string2features import string2features

ppl = make_pipeline(FeatureUnion([('aif', AddImgFeatures()), 
                                  ('af', AddFeatures()),
                                  ('cv', make_pipeline(
                                      string2features(),
                                      CountVectorizer(token_pattern=r'(?u)\b\w+\b')))]), 
                     LogisticRegression())

Обучаем модель:
логистическая регрессия, для каждой строки - тип строки + уровень???

как работать с уровнем? строка-название + строка-уровень

In [ ]:
import json
with open("file_train2.json", "r") as read_file:
    doc_with_labels = json.load(read_file)

In [ ]:
x1 = [doc_with_labels[0]]
aif = AddImgFeatures()
print(aif.transform(x1))
af = AddFeatures()
print(af.transform(x1))

Обучаем и сохраняем обученную модель

In [ ]:
import pickle as pkl
clf = ppl.fit(X, y)
pkl.dump(clf, open("model.pkl", "wb"))

Кросс-валидация

In [8]:
import json
with open("file_with_labels2.json", "r") as read_file:
    doc_test = json.load(read_file)
    y = [x["label"] for x in doc_test]
    X = [{"text": x["text"], "bbox": x["bbox"], "name": x["name"]} for x in doc_test]

https://neurohive.io/ru/osnovy-data-science/gradientyj-busting/

In [37]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score


clf = make_pipeline(FeatureUnion([
                                  ('af', AddFeatures()),
                                  ('cv', make_pipeline(
                                      string2features(),
                                      CountVectorizer(token_pattern=r'(?u)\b\w+\b')))]), 
                     GradientBoostingClassifier())

groups = [x["name"] for x in doc_test]
group_kfold = GroupKFold(n_splits=2)
group_kfold.get_n_splits(X, y, groups)
scores = []
for train_index, test_index in group_kfold.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    y_test = clf.predict(X_test)
    scores.append(f1_score(y_test, y_pred, average='macro'))

TypeError: Singleton array array(249) cannot be considered a valid collection.

без aif 0.37956220744365304

c aif 0.37134152944319837

с нормализованной aif 0.3825286683816205

GradientBoostingClassifier с нормализованной aif 0.7602626622815247

без aif 0.41892482779902956

In [13]:
import numpy as np
score = np.mean(scores['test_score'])
score

0.7602626622815247

1) размечалка + манифест +

2) признаки на основе предыдущих (следующих) строк ?

3) countvectorizer по первому второму словам +

4) признак - средний размер баундин бокса +

5) gboost +, randomforest (1000 trees) ?

6) жирность

7) кросс-валидация по группам

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html

In [14]:
from sklearn.metrics import cohen_kappa_score

with open("file_with_labels2.json", "r") as read_file:
    labeled_doc = json.load(read_file)
with open("file_with_labels2_ilya.json", "r") as read_file:
    labeled_doc_ilya = json.load(read_file)

In [17]:
from functools import cmp_to_key

def cmp(x, y):
    if x['name'] == y['name']:
        if x['bbox'][1] < y['bbox'][1]:
            return -1
        else:
            return 1
    elif x['name'] < y['name']:
        return -1 
    else:
        return 1

In [18]:
labeled_doc.sort(key=cmp_to_key(cmp))

In [19]:
labeled_doc

[{'label': 3,
  'name': '0013.jpeg',
  'text': 'СТО 1.1.1.01.0678-2015',
  'bbox': [838, 59, 194, 13]},
 {'label': 3,
  'name': '0013.jpeg',
  'text': 'производства и потребления и количество забираемой из водных объектов и',
  'bbox': [112, 114, 986, 25]},
 {'label': 3,
  'name': '0013.jpeg',
  'text': 'сбрасываемой воды.',
  'bbox': [111, 146, 233, 25]},
 {'label': 2,
  'name': '0013.jpeg',
  'text': '7.10.14 Метрологическое обеспечение средств контроля выбросов и сбросов',
  'bbox': [180, 177, 918, 25]},
 {'label': 3,
  'name': '0013.jpeg',
  'text': 'должно осуществляться в соответствии © 5.9.',
  'bbox': [111, 204, 516, 32]},
 {'label': 1,
  'name': '0013.jpeg',
  'text': '7.11 Физическая защита ядерных материалов, ядерных установок и',
  'bbox': [180, 263, 919, 26]},
 {'label': 1,
  'name': '0013.jpeg',
  'text': 'пунктов хранения ядерных материалов на АС',
  'bbox': [111, 295, 570, 24]},
 {'label': 2,
  'name': '0013.jpeg',
  'text': '7.11.1 Физическая защита ядерных материалов,

In [20]:
labeled_doc_ilya.sort(key=cmp_to_key(cmp))

In [21]:
labeled_doc_ilya

[{'label': 3,
  'name': '0013.jpeg',
  'text': 'СТО 1.1.1.01.0678-2015',
  'bbox': [838, 59, 194, 13]},
 {'label': 3,
  'name': '0013.jpeg',
  'text': 'производства и потребления и количество забираемой из водных объектов и',
  'bbox': [112, 114, 986, 25]},
 {'label': 3,
  'name': '0013.jpeg',
  'text': 'сбрасываемой воды.',
  'bbox': [111, 146, 233, 25]},
 {'label': 2,
  'name': '0013.jpeg',
  'text': '7.10.14 Метрологическое обеспечение средств контроля выбросов и сбросов',
  'bbox': [180, 177, 918, 25]},
 {'label': 3,
  'name': '0013.jpeg',
  'text': 'должно осуществляться в соответствии © 5.9.',
  'bbox': [111, 208, 522, 25]},
 {'label': 1,
  'name': '0013.jpeg',
  'text': '7.11 Физическая защита ядерных материалов, ядерных установок и',
  'bbox': [180, 263, 919, 26]},
 {'label': 1,
  'name': '0013.jpeg',
  'text': 'пунктов хранения ядерных материалов на АС',
  'bbox': [111, 295, 570, 24]},
 {'label': 2,
  'name': '0013.jpeg',
  'text': '7.11.1 Физическая защита ядерных материалов,

In [27]:
labels1 = [x["label"] for x in labeled_doc]
labels2 = [x["label"] for x in labeled_doc_ilya]

In [29]:
cohen_kappa_score(labels1, labels2, labels=[1, 2, 3, 4])

0.9758103819225694

In [35]:
d = {}
for i, elem in enumerate(labeled_doc):
    d[(tuple(elem['bbox']), elem['name'])] = [labels1[i], labels2[i]]
d

{((838, 59, 194, 13), '0013.jpeg'): [3, 3],
 ((112, 114, 986, 25), '0013.jpeg'): [3, 3],
 ((111, 146, 233, 25), '0013.jpeg'): [3, 3],
 ((180, 177, 918, 25), '0013.jpeg'): [2, 2],
 ((111, 204, 516, 32), '0013.jpeg'): [3, 3],
 ((180, 263, 919, 26), '0013.jpeg'): [1, 1],
 ((111, 295, 570, 24), '0013.jpeg'): [1, 1],
 ((180, 337, 919, 25), '0013.jpeg'): [2, 2],
 ((111, 368, 988, 26), '0013.jpeg'): [3, 3],
 ((111, 399, 986, 25), '0013.jpeg'): [3, 3],
 ((111, 437, 989, 19), '0013.jpeg'): [3, 3],
 ((111, 462, 987, 24), '0013.jpeg'): [3, 3],
 ((111, 494, 378, 24), '0013.jpeg'): [3, 3],
 ((180, 524, 918, 25), '0013.jpeg'): [2, 2],
 ((111, 557, 988, 24), '0013.jpeg'): [3, 3],
 ((111, 593, 357, 19), '0013.jpeg'): [3, 3],
 ((180, 618, 919, 25), '0013.jpeg'): [2, 2],
 ((111, 649, 920, 25), '0013.jpeg'): [3, 3],
 ((180, 684, 622, 23), '0013.jpeg'): [2, 2],
 ((180, 716, 762, 25), '0013.jpeg'): [2, 2],
 ((180, 748, 821, 26), '0013.jpeg'): [2, 2],
 ((180, 783, 919, 24), '0013.jpeg'): [2, 2],
 ((111, 814

In [36]:
for item in d.items():
    if (len(item[1]) == 1) or (len(item[1]) == 2 
                               and item[1][0] != item[1][1]):
        img = cv2.imread('different_docs/' + item[0][1])
        (x, y, w, h) = item[0][0]
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.imwrite('different_docs/' + item[0][1], img)